<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/rlace_sd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q git+https://github.com/huggingface/peft.git@main
!pip install -U accelerate
!pip install evaluate
!pip install jiwer
!pip install --upgrade transformers bitsandbytes datasets torch torchvision torchaudio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 11.4 MB

In [ ]:
import torch
import numpy as np
from datasets import load_dataset, DatasetDict
from transformers import (
    WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration,
    BitsAndBytesConfig, Trainer, TrainingArguments, AutoTokenizer
)
from torch.utils.data import DataLoader
from accelerate import Accelerator
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from torch.optim import AdamW
import torch.nn.functional as F
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate
import re

# Step 1: Load VoxPopuli Dataset
language = "Estonian"
voxpopuli = DatasetDict()
voxpopuli["train"] = load_dataset("facebook/voxpopuli", "sl", split="train", trust_remote_code=True)
voxpopuli["validation"] = load_dataset("facebook/voxpopuli", "sl", split="validation", trust_remote_code=True)
voxpopuli["test"] = load_dataset("facebook/voxpopuli", "sl", split="test", trust_remote_code=True)

# Sampling
#voxpopuli["train"] = voxpopuli["train"].select(range(800))
#voxpopuli["validation"] = voxpopuli["validation"].select(range(300))
#voxpopuli["test"] = voxpopuli["test"].select(range(300))

# Step 2: Load Whisper Model & Processor
model_name_or_path = "openai/whisper-tiny"
#quantization_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_enable_fp32_cpu_offload=True)
#accelerator = Accelerator(mixed_precision="fp16")
model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path)
#model = accelerator.prepare(model)

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)
tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task="transcribe")
processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task="transcribe")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

voxpopuli.py:   0%|          | 0.00/8.84k [00:00<?, ?B/s]

n_files.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

asr_train.tsv:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [ ]:
voxpopuli_dataset = voxpopuli.remove_columns(['audio_id', 'language', 'raw_text', 'speaker_id', 'is_gold_transcript', 'accent'])
max_label_length = model.config.max_length
min_input_length = 0.0
max_input_length = 30.0

In [ ]:
# Data Preparation Function
def prepare_dataset(batch):
    batch_input_features = []
    batch_labels = []
    batch_genders = []

    for idx in range(len(batch["audio"])):
        audio_features = feature_extractor(batch["audio"][idx]["array"], sampling_rate=batch["audio"][idx]["sampling_rate"]).input_features
        batch_input_features.extend(audio_features)

        label = tokenizer(batch["normalized_text"][idx]).input_ids
        batch_labels.append(label)

        # Assuming 'gender' field exists and contains strings like "male" or "female"
        gender = 1 if batch["gender"][idx] == "male" else 2  # Map "male" to 1, "female" to 2
        batch_genders.append(gender)

    return {"input_features": batch_input_features, "labels": batch_labels, "groups": batch_genders}

voxpopuli_data = voxpopuli_dataset.map(prepare_dataset, remove_columns=voxpopuli_dataset.column_names["train"], batched=True)

Map:   0%|          | 0/2099 [00:00<?, ? examples/s]

Map:   0%|          | 0/889 [00:00<?, ? examples/s]

Map:   0%|          | 0/309 [00:00<?, ? examples/s]

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, features):
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        if labels.size(1) > 0 and (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]
        batch["labels"] = labels

        # Directly use 'groups' from features, assuming they are already numeric
        batch["groups"] = torch.tensor([feature["groups"] for feature in features], dtype=torch.long)

        return batch

from torch.utils.data import DataLoader
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from transformers import Trainer, TrainingArguments, WhisperForConditionalGeneration
from datasets import load_dataset  # if you're using HuggingFace Datasets

##############################################
# Part 1: Compute R-LACE Projection Matrix (P)
##############################################
def r_lace_gpu(X, y, k=1, device="cuda"):
    """
    Compute a projection matrix to remove up to k gender directions based on
    the difference between mean male and mean female embeddings.

    NOTE: Since we compute a single difference vector, its rank = 1,
    so only k=1 effectively removes nonzero gender information.
    """
    X = X.to(device)
    y = y.to(device)
    # Center the embeddings
    X_centered = X - X.mean(dim=0, keepdim=True)
    # In our data, y == 1 => male, y == 2 => female.
    mean_male   = X_centered[y == 1].mean(dim=0)  # shape: [d]
    mean_female = X_centered[y == 2].mean(dim=0)  # shape: [d]
    # Difference vector
    gender_difference = mean_male - mean_female   # shape: [d]
    # Reshape to [1, d] for SVD
    difference_mat = gender_difference.unsqueeze(0)  # shape: [1, d]
    U, S, Vt = torch.linalg.svd(difference_mat, full_matrices=False)
    gender_directions = Vt[:k]  # shape: [k, d]
    V = gender_directions.T     # shape: [d, k]
    P = torch.eye(X.shape[1], device=device) - V @ V.T  # shape: [d, d]
    return P

# Prepare device
device = "cuda" if torch.cuda.is_available() else "cpu"

# --- Gather training features for computing P ---
# Here we assume voxpopuli_data["train"] is a dataset (e.g., HuggingFace Dataset)
X_train_list = []
y_train_list = []
for ex in voxpopuli_data["train"]:
    # ex["input_features"]: shape [80, T] (or [T, 80])
    feat = np.array(ex["input_features"])
    # Assume mel bins along axis 0; pool over time (axis=1)
    feat_mean = feat.mean(axis=1) if feat.shape[0] == 80 else feat.mean(axis=0)
    X_train_list.append(feat_mean)  # Each is [80]
    y_train_list.append(ex["groups"])  # groups: 1 (male) or 2 (female)

X_train_mean_np = np.array(X_train_list)  # shape: [N_train, 80]
y_train_np = np.array(y_train_list)         # shape: [N_train]

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train_mean_np, dtype=torch.float32, device=device)
y_train_tensor = torch.tensor(y_train_np, dtype=torch.long, device=device)

# Compute projection matrix P (only k=1 is effective)
k = 1
P = r_lace_gpu(X_train_tensor, y_train_tensor, k=k, device=device)
print("Projection matrix P computed.")


def apply_debias(example, P, device="cuda"):
    """
    Apply the projection matrix P to the input_features.
    Assumes input_features is shape [80, T] (frequency x time) or [T, 80].
    The projection is applied along the frequency dimension.
    """
    feat = np.array(example["input_features"])
    if feat.shape[0] == 80:  # shape: [80, T]
        feat_tensor = torch.tensor(feat, dtype=torch.float32, device=device)
        debiased = P @ feat_tensor
    else:
        feat_tensor = torch.tensor(feat, dtype=torch.float32, device=device)
        debiased = feat_tensor @ P.T
    example["input_features"] = debiased.cpu().numpy().tolist()
    return example

# Update train and validation datasets
debiased_train = voxpopuli_data["train"].map(lambda ex: apply_debias(ex, P, device=device))
debiased_val = voxpopuli_data["validation"].map(lambda ex: apply_debias(ex, P, device=device))
print("Datasets updated with debiased input features.")

Projection matrix P computed.


Map:   0%|          | 0/2099 [00:00<?, ? examples/s]

Map:   0%|          | 0/889 [00:00<?, ? examples/s]

Datasets updated with debiased input features.


In [ ]:
class DataCollatorForDebiasedSeq2Seq:
    def __call__(self, features):
        # Process "input_features": assumed to be lists with shape [80, T]
        input_feats = [torch.tensor(f["input_features"], dtype=torch.float32) for f in features]
        max_time_feat = max(feat.shape[1] for feat in input_feats)
        padded_feats = []
        for feat in input_feats:
            t = feat.shape[1]
            if t < max_time_feat:
                pad_amount = max_time_feat - t
                feat_padded = F.pad(feat, (0, pad_amount), "constant", 0.0)
            else:
                feat_padded = feat
            padded_feats.append(feat_padded)
        batch = {"input_features": torch.stack(padded_feats)}

        # Process "labels": assume they are lists of ints (variable length)
        if "labels" in features[0]:
            labels_list = [torch.tensor(f["labels"], dtype=torch.long) for f in features]
            max_len_labels = max(l.shape[0] for l in labels_list)
            padded_labels = []
            for l in labels_list:
                if l.shape[0] < max_len_labels:
                    pad_amount = max_len_labels - l.shape[0]
                    # Pad with -100 as ignore_index for cross-entropy loss
                    l_padded = F.pad(l, (0, pad_amount), "constant", -100)
                else:
                    l_padded = l
                padded_labels.append(l_padded)
            batch["labels"] = torch.stack(padded_labels)

        # Process "groups": assume a scalar per example
        if "groups" in features[0]:
            batch["groups"] = torch.tensor([f["groups"] for f in features], dtype=torch.long)

        # Process other fields if needed...
        return batch

In [ ]:
def compute_spectral_norm(model):
    """Compute the spectral norm of the model's parameters."""
    spectral_norm = 0.0
    for p in model.parameters():
        if p.dtype.is_floating_point:
            spectral_norm += torch.linalg.norm(p).item()
    return spectral_norm

class SpectralDecouplingTrainer(Trainer):
    def __init__(self, model, lambda_spectral=0.006, *args, **kwargs):
        super().__init__(model, *args, **kwargs)
        self.lambda_spectral = lambda_spectral

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        input_features = inputs["input_features"].to(device)
        labels = inputs["labels"].to(device)

        # Forward pass
        outputs = model(input_features=input_features, labels=labels)
        loss = outputs.loss

        # Compute Spectral Decoupling Loss
        spectral_norm = compute_spectral_norm(model)
        spectral_decoupling_loss = self.lambda_spectral * spectral_norm

        # Total loss
        total_loss = loss + spectral_decoupling_loss

        return (total_loss, outputs) if return_outputs else total_loss

In [ ]:
##############################################
# Part 5: Training Setup and Start Training
##############################################
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=6,
    weight_decay=0.01,
    save_total_limit=2,
    gradient_accumulation_steps=8,
    fp16=True,
    remove_unused_columns=False,
    logging_strategy="steps",
    logging_steps=10,
    report_to="none",
    max_grad_norm=1.0
)

trainer = SpectralDecouplingTrainer(
    model=model,
    args=training_args,
    train_dataset=debiased_train,
    eval_dataset=debiased_val,
    data_collator=DataCollatorForDebiasedSeq2Seq()
)
print("Starting training with sd loss on debiased (R-LACE) features...")
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting training with IRM loss on debiased (R-LACE) features...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
0,27.723900,27.573410
1,27.348800,27.281742
2,26.942900,27.022949
3,26.885500,26.970617
4,26.796700,26.946526
5,26.814000,26.938559


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=390, training_loss=27.34190916403746, metrics={'train_runtime': 2278.2559, 'train_samples_per_second': 5.528, 'train_steps_per_second': 0.171, 'total_flos': 3.087208194048e+17, 'train_loss': 27.34190916403746, 'epoch': 5.9904761904761905})

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import evaluate
import re
import random
from torch.utils.data import DataLoader

# Ensure WER metric is loaded
wer_metric = evaluate.load("wer")

#   - "groups": the gender label (1 for male, 2 for female)
debiased_test = voxpopuli_data["test"].map(lambda ex: apply_debias(ex, P, device="cuda"))
# (Optionally, you can further process debiased_test with your custom create_hf_dataset function.)

class DataCollatorForDebiasedSeq2Seq:
    def __call__(self, features):
        # Process "input_features": each is a 2D array [80, T]
        input_feats = [torch.tensor(f["input_features"], dtype=torch.float32) for f in features]
        max_time = max(feat.shape[1] for feat in input_feats)
        padded_feats = []
        for feat in input_feats:
            t = feat.shape[1]
            if t < max_time:
                pad_amount = max_time - t
                feat_padded = F.pad(feat, (0, pad_amount), "constant", 0.0)
            else:
                feat_padded = feat
            padded_feats.append(feat_padded)
        batch = {"input_features": torch.stack(padded_feats)}

        # Process "labels": assume they are 1D arrays (variable length)
        if "labels" in features[0]:
            labels_list = [torch.tensor(f["labels"], dtype=torch.long) for f in features]
            max_len_labels = max(l.shape[0] for l in labels_list)
            padded_labels = []
            for l in labels_list:
                if l.shape[0] < max_len_labels:
                    pad_amount = max_len_labels - l.shape[0]
                    l_padded = F.pad(l, (0, pad_amount), "constant", -100)
                else:
                    l_padded = l
                padded_labels.append(l_padded)
            batch["labels"] = torch.stack(padded_labels)

        # Process "groups": assume scalar per example
        if "groups" in features[0]:
            batch["groups"] = torch.tensor([f["groups"] for f in features], dtype=torch.long)

        return batch

# Create a data collator instance
data_collator = DataCollatorForDebiasedSeq2Seq()
eval_dataloader = DataLoader(debiased_test, batch_size=4, collate_fn=data_collator)

def normalize_text(text):
    text = text.lower().strip()
    text = text.replace("\n", " ")
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Remove non-alphanumeric characters
    return text

def evaluate_wer_debiased(eval_dataloader, model, processor, device):
    model.eval()  # Set model to evaluation mode

    total_wer, total_male_wer, total_female_wer = 0, 0, 0
    male_count, female_count, total_count = 0, 0, 0

    with torch.no_grad():  # Disable gradients for evaluation
        for batch in eval_dataloader:
            input_features = batch['input_features'].to(device)
            labels = batch['labels'].to(device)
            groups = batch['groups'].to(device)

            # Create an attention mask: assume nonzero values in input_features indicate valid data
            attention_mask = (input_features.sum(dim=1) != 0).long()
            # Generate predictions (adjust max_new_tokens and num_beams as needed)
            generated_ids = model.generate(
                input_features,
                max_new_tokens=256,
                num_beams=1,
                attention_mask=attention_mask,
                language="sl"
            )

            # Decode predictions
            transcriptions = processor.batch_decode(generated_ids, skip_special_tokens=True)

            for idx, (transcription, label_ids, group) in enumerate(zip(transcriptions, labels, groups)):
                hypothesis_text = normalize_text(transcription)

                # Convert label_ids to list and filter out invalid token IDs (padding -100)
                label_ids = label_ids.tolist()
                label_ids = [tid for tid in label_ids if tid >= 0 and tid < processor.tokenizer.vocab_size]

                if label_ids:
                    try:
                        reference_text = normalize_text(processor.tokenizer.decode(label_ids, skip_special_tokens=True))
                    except Exception as e:
                        continue

                    # Compute WER for the sample
                    sample_wer = wer_metric.compute(predictions=[hypothesis_text], references=[reference_text])

                    total_wer += sample_wer
                    total_count += 1

                    if group.item() == 1:  # Male
                        total_male_wer += sample_wer
                        male_count += 1
                    elif group.item() == 2:  # Female
                        total_female_wer += sample_wer
                        female_count += 1

    avg_wer = total_wer / total_count if total_count > 0 else 0
    avg_male_wer = total_male_wer / male_count if male_count > 0 else 0
    avg_female_wer = total_female_wer / female_count if female_count > 0 else 0

    return avg_wer, avg_male_wer, avg_female_wer

# ------------------------------------------------------------------------------
# Step 5: Run Evaluation
# ------------------------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

avg_wer_debiased, avg_male_wer_debiased, avg_female_wer_debiased = evaluate_wer_debiased(
    eval_dataloader, model, processor, device
)

print(f"\nDebiased Combined Average WER: {avg_wer_debiased:.4f}")
print(f"Debiased Male Average WER: {avg_male_wer_debiased:.4f}")
print(f"Debiased Female Average WER: {avg_female_wer_debiased:.4f}")
print(f"Debiased Gender WER Gap: {abs(avg_male_wer_debiased - avg_female_wer_debiased):.4f}")

Map:   0%|          | 0/309 [00:00<?, ? examples/s]

You have passed language=sl, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=sl.



Debiased Combined Average WER: 0.7199
Debiased Male Average WER: 0.7872
Debiased Female Average WER: 0.6015
Debiased Gender WER Gap: 0.1856


In [ ]:
import torch.nn.functional as F

def compute_erm_loss(logits, labels):
    # Flatten the logits and labels for cross-entropy computation
    logits_flat = logits.view(-1, logits.size(-1))  # Shape: (batch_size * sequence_length, vocab_size)
    labels_flat = labels.view(-1)  # Shape: (batch_size * sequence_length)

    # Ignore padding tokens (label = -100)
    loss = F.cross_entropy(logits_flat, labels_flat, ignore_index=-100)
    return loss

class ERMTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        input_features = inputs["input_features"].to(device)
        labels = inputs["labels"].to(device)

        # Forward pass
        outputs = model(input_features=input_features, labels=labels)
        logits = outputs.logits

        # Explicitly compute ERM loss
        erm_loss = compute_erm_loss(logits, labels)

        return (erm_loss, outputs) if return_outputs else erm_loss

In [ ]:
##############################################
# Part 5: Training Setup and Start Training
##############################################
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=6,
    weight_decay=0.01,
    save_total_limit=2,
    gradient_accumulation_steps=8,
    fp16=True,
    remove_unused_columns=False,
    logging_strategy="steps",
    logging_steps=10,
    report_to="none",
    max_grad_norm=1.0
)

trainer = ERMTrainer(
    model=model,
    args=training_args,
    train_dataset=debiased_train,
    eval_dataset=debiased_val,
    data_collator=DataCollatorForDebiasedSeq2Seq()
)
print("Starting training with erm loss on debiased (R-LACE) features...")
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting training with erm loss on debiased (R-LACE) features...


Epoch,Training Loss,Validation Loss
0,0.785200,0.918419
1,0.724100,0.882773
2,0.585800,0.865278
3,0.584100,0.855476
4,0.524200,0.852151
5,0.549900,0.850552


TrainOutput(global_step=390, training_loss=0.638248052352514, metrics={'train_runtime': 2292.5252, 'train_samples_per_second': 5.494, 'train_steps_per_second': 0.17, 'total_flos': 3.087208194048e+17, 'train_loss': 0.638248052352514, 'epoch': 5.9904761904761905})

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import evaluate
import re
import random
from torch.utils.data import DataLoader

# Ensure WER metric is loaded
wer_metric = evaluate.load("wer")

#   - "groups": the gender label (1 for male, 2 for female)
debiased_test = voxpopuli_data["test"].map(lambda ex: apply_debias(ex, P, device="cuda"))
# (Optionally, you can further process debiased_test with your custom create_hf_dataset function.)

class DataCollatorForDebiasedSeq2Seq:
    def __call__(self, features):
        # Process "input_features": each is a 2D array [80, T]
        input_feats = [torch.tensor(f["input_features"], dtype=torch.float32) for f in features]
        max_time = max(feat.shape[1] for feat in input_feats)
        padded_feats = []
        for feat in input_feats:
            t = feat.shape[1]
            if t < max_time:
                pad_amount = max_time - t
                feat_padded = F.pad(feat, (0, pad_amount), "constant", 0.0)
            else:
                feat_padded = feat
            padded_feats.append(feat_padded)
        batch = {"input_features": torch.stack(padded_feats)}

        # Process "labels": assume they are 1D arrays (variable length)
        if "labels" in features[0]:
            labels_list = [torch.tensor(f["labels"], dtype=torch.long) for f in features]
            max_len_labels = max(l.shape[0] for l in labels_list)
            padded_labels = []
            for l in labels_list:
                if l.shape[0] < max_len_labels:
                    pad_amount = max_len_labels - l.shape[0]
                    l_padded = F.pad(l, (0, pad_amount), "constant", -100)
                else:
                    l_padded = l
                padded_labels.append(l_padded)
            batch["labels"] = torch.stack(padded_labels)

        # Process "groups": assume scalar per example
        if "groups" in features[0]:
            batch["groups"] = torch.tensor([f["groups"] for f in features], dtype=torch.long)

        return batch

# Create a data collator instance
data_collator = DataCollatorForDebiasedSeq2Seq()
eval_dataloader = DataLoader(debiased_test, batch_size=4, collate_fn=data_collator)

def normalize_text(text):
    text = text.lower().strip()
    text = text.replace("\n", " ")
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Remove non-alphanumeric characters
    return text

def evaluate_wer_debiased(eval_dataloader, model, processor, device):
    model.eval()  # Set model to evaluation mode

    total_wer, total_male_wer, total_female_wer = 0, 0, 0
    male_count, female_count, total_count = 0, 0, 0

    with torch.no_grad():  # Disable gradients for evaluation
        for batch in eval_dataloader:
            input_features = batch['input_features'].to(device)
            labels = batch['labels'].to(device)
            groups = batch['groups'].to(device)

            # Create an attention mask: assume nonzero values in input_features indicate valid data
            attention_mask = (input_features.sum(dim=1) != 0).long()
            # Generate predictions (adjust max_new_tokens and num_beams as needed)
            generated_ids = model.generate(
                input_features,
                max_new_tokens=256,
                num_beams=1,
                attention_mask=attention_mask,
                language="sl"
            )

            # Decode predictions
            transcriptions = processor.batch_decode(generated_ids, skip_special_tokens=True)

            for idx, (transcription, label_ids, group) in enumerate(zip(transcriptions, labels, groups)):
                hypothesis_text = normalize_text(transcription)

                # Convert label_ids to list and filter out invalid token IDs (padding -100)
                label_ids = label_ids.tolist()
                label_ids = [tid for tid in label_ids if tid >= 0 and tid < processor.tokenizer.vocab_size]

                if label_ids:
                    try:
                        reference_text = normalize_text(processor.tokenizer.decode(label_ids, skip_special_tokens=True))
                    except Exception as e:
                        continue

                    # Compute WER for the sample
                    sample_wer = wer_metric.compute(predictions=[hypothesis_text], references=[reference_text])

                    total_wer += sample_wer
                    total_count += 1

                    if group.item() == 1:  # Male
                        total_male_wer += sample_wer
                        male_count += 1
                    elif group.item() == 2:  # Female
                        total_female_wer += sample_wer
                        female_count += 1

    avg_wer = total_wer / total_count if total_count > 0 else 0
    avg_male_wer = total_male_wer / male_count if male_count > 0 else 0
    avg_female_wer = total_female_wer / female_count if female_count > 0 else 0

    return avg_wer, avg_male_wer, avg_female_wer

# ------------------------------------------------------------------------------
# Step 5: Run Evaluation
# ------------------------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

avg_wer_debiased, avg_male_wer_debiased, avg_female_wer_debiased = evaluate_wer_debiased(
    eval_dataloader, model, processor, device
)

print(f"\nDebiased Combined Average WER: {avg_wer_debiased:.4f}")
print(f"Debiased Male Average WER: {avg_male_wer_debiased:.4f}")
print(f"Debiased Female Average WER: {avg_female_wer_debiased:.4f}")
print(f"Debiased Gender WER Gap: {abs(avg_male_wer_debiased - avg_female_wer_debiased):.4f}")


Debiased Combined Average WER: 0.7504
Debiased Male Average WER: 0.8025
Debiased Female Average WER: 0.6587
Debiased Gender WER Gap: 0.1438


In [ ]:
def compute_dro_loss(losses, groups):
    # Expand groups to match the shape of losses
    # Each example in the batch has a sequence of tokens, so we need to repeat the group ID for each token
    groups_expanded = groups.repeat_interleave(losses.size(0) // groups.size(0))

    unique_groups = torch.unique(groups_expanded)
    subgroup_losses = []

    for group in unique_groups:
        # Mask for the current group
        group_mask = (groups_expanded == group)
        if group_mask.sum() > 0:  # Ensure the group is not empty
            subgroup_loss = losses[group_mask].mean()  # Mean loss for the subgroup
            subgroup_losses.append(subgroup_loss)

    # Return the maximum subgroup loss (worst-case loss)
    dro_loss = torch.max(torch.stack(subgroup_losses))
    return dro_loss

class DROTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        input_features = inputs["input_features"].to(device)
        labels = inputs["labels"].to(device)
        groups = inputs["groups"].to(device)

        # Forward pass
        outputs = model(input_features=input_features, labels=labels)
        logits = outputs.logits

        # Compute the per-example cross-entropy loss
        loss_fct = torch.nn.CrossEntropyLoss(reduction="none")
        losses = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))

        # Expand groups to match the shape of losses
        # Each example in the batch has a sequence of tokens, so we need to repeat the group ID for each token
        groups_expanded = groups.repeat_interleave(losses.size(0) // groups.size(0))

        # Compute DRO loss (worst-case subgroup loss)
        dro_loss = compute_dro_loss(losses, groups_expanded)

        return (dro_loss, outputs) if return_outputs else dro_loss


In [ ]:
# Training setup
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=6,
    weight_decay=0.01,
    save_total_limit=2,
    gradient_accumulation_steps=8,
    fp16=True,
    remove_unused_columns=False,
    logging_strategy="steps",
    logging_steps=10,
    report_to="none",
    max_grad_norm=1.0
)

# Initialize DRO Trainer
trainer = DROTrainer(
    model=model,
    args=training_args,
    train_dataset=debiased_train,
    eval_dataset=debiased_val,
    data_collator=DataCollatorForDebiasedSeq2Seq()
)

# Start training with DRO
print("Starting training with DRO loss...")
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting training with DRO loss...


Epoch,Training Loss,Validation Loss
0,0.467500,0.719828
1,0.433300,0.715167
2,0.371300,0.712278
3,0.362600,0.712694
4,0.329300,0.714114
5,0.357700,0.713488


TrainOutput(global_step=390, training_loss=0.3953675422912989, metrics={'train_runtime': 2290.9823, 'train_samples_per_second': 5.497, 'train_steps_per_second': 0.17, 'total_flos': 3.087208194048e+17, 'train_loss': 0.3953675422912989, 'epoch': 5.9904761904761905})

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import evaluate
import re
import random
from torch.utils.data import DataLoader

# Ensure WER metric is loaded
wer_metric = evaluate.load("wer")

#   - "groups": the gender label (1 for male, 2 for female)
debiased_test = voxpopuli_data["test"].map(lambda ex: apply_debias(ex, P, device="cuda"))
# (Optionally, you can further process debiased_test with your custom create_hf_dataset function.)

class DataCollatorForDebiasedSeq2Seq:
    def __call__(self, features):
        # Process "input_features": each is a 2D array [80, T]
        input_feats = [torch.tensor(f["input_features"], dtype=torch.float32) for f in features]
        max_time = max(feat.shape[1] for feat in input_feats)
        padded_feats = []
        for feat in input_feats:
            t = feat.shape[1]
            if t < max_time:
                pad_amount = max_time - t
                feat_padded = F.pad(feat, (0, pad_amount), "constant", 0.0)
            else:
                feat_padded = feat
            padded_feats.append(feat_padded)
        batch = {"input_features": torch.stack(padded_feats)}

        # Process "labels": assume they are 1D arrays (variable length)
        if "labels" in features[0]:
            labels_list = [torch.tensor(f["labels"], dtype=torch.long) for f in features]
            max_len_labels = max(l.shape[0] for l in labels_list)
            padded_labels = []
            for l in labels_list:
                if l.shape[0] < max_len_labels:
                    pad_amount = max_len_labels - l.shape[0]
                    l_padded = F.pad(l, (0, pad_amount), "constant", -100)
                else:
                    l_padded = l
                padded_labels.append(l_padded)
            batch["labels"] = torch.stack(padded_labels)

        # Process "groups": assume scalar per example
        if "groups" in features[0]:
            batch["groups"] = torch.tensor([f["groups"] for f in features], dtype=torch.long)

        return batch

# Create a data collator instance
data_collator = DataCollatorForDebiasedSeq2Seq()
eval_dataloader = DataLoader(debiased_test, batch_size=4, collate_fn=data_collator)

def normalize_text(text):
    text = text.lower().strip()
    text = text.replace("\n", " ")
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Remove non-alphanumeric characters
    return text

def evaluate_wer_debiased(eval_dataloader, model, processor, device):
    model.eval()  # Set model to evaluation mode

    total_wer, total_male_wer, total_female_wer = 0, 0, 0
    male_count, female_count, total_count = 0, 0, 0

    with torch.no_grad():  # Disable gradients for evaluation
        for batch in eval_dataloader:
            input_features = batch['input_features'].to(device)
            labels = batch['labels'].to(device)
            groups = batch['groups'].to(device)

            # Create an attention mask: assume nonzero values in input_features indicate valid data
            attention_mask = (input_features.sum(dim=1) != 0).long()
            # Generate predictions (adjust max_new_tokens and num_beams as needed)
            generated_ids = model.generate(
                input_features,
                max_new_tokens=256,
                num_beams=1,
                attention_mask=attention_mask,
                language="sl"
            )

            # Decode predictions
            transcriptions = processor.batch_decode(generated_ids, skip_special_tokens=True)

            for idx, (transcription, label_ids, group) in enumerate(zip(transcriptions, labels, groups)):
                hypothesis_text = normalize_text(transcription)

                # Convert label_ids to list and filter out invalid token IDs (padding -100)
                label_ids = label_ids.tolist()
                label_ids = [tid for tid in label_ids if tid >= 0 and tid < processor.tokenizer.vocab_size]

                if label_ids:
                    try:
                        reference_text = normalize_text(processor.tokenizer.decode(label_ids, skip_special_tokens=True))
                    except Exception as e:
                        continue

                    # Compute WER for the sample
                    sample_wer = wer_metric.compute(predictions=[hypothesis_text], references=[reference_text])

                    total_wer += sample_wer
                    total_count += 1

                    if group.item() == 1:  # Male
                        total_male_wer += sample_wer
                        male_count += 1
                    elif group.item() == 2:  # Female
                        total_female_wer += sample_wer
                        female_count += 1

    avg_wer = total_wer / total_count if total_count > 0 else 0
    avg_male_wer = total_male_wer / male_count if male_count > 0 else 0
    avg_female_wer = total_female_wer / female_count if female_count > 0 else 0

    return avg_wer, avg_male_wer, avg_female_wer

# ------------------------------------------------------------------------------
# Step 5: Run Evaluation
# ------------------------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

avg_wer_debiased, avg_male_wer_debiased, avg_female_wer_debiased = evaluate_wer_debiased(
    eval_dataloader, model, processor, device
)

print(f"\nDebiased Combined Average WER: {avg_wer_debiased:.4f}")
print(f"Debiased Male Average WER: {avg_male_wer_debiased:.4f}")
print(f"Debiased Female Average WER: {avg_female_wer_debiased:.4f}")
print(f"Debiased Gender WER Gap: {abs(avg_male_wer_debiased - avg_female_wer_debiased):.4f}")


Debiased Combined Average WER: 0.7559
Debiased Male Average WER: 0.8255
Debiased Female Average WER: 0.6333
Debiased Gender WER Gap: 0.1922


In [ ]:
import torch
import torch.nn.functional as F

# ERM Loss (standard cross-entropy loss)
def compute_erm_loss(logits, labels):
    loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
    erm_loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
    return erm_loss

# Corrected IRM Penalty
def irm_penalty(logits, labels, groups):
    penalty = 0.0
    unique_groups = torch.unique(groups)

    for group in unique_groups:
        mask = groups == group
        if mask.sum() > 1:
            logits_group = logits[mask]
            labels_group = labels[mask]

            scale = torch.tensor(1.0, device=logits.device, requires_grad=True)
            loss_group = F.cross_entropy(logits_group.view(-1, logits.size(-1)) * scale, labels_group.view(-1), ignore_index=-100)
            grad = torch.autograd.grad(loss_group, [scale], create_graph=True)[0]
            penalty += torch.sum(grad**2)

    penalty /= len(unique_groups)
    return penalty

# Corrected Spectral Norm
def compute_spectral_norm(model):
    spectral_norm = torch.tensor(0.0, device=next(model.parameters()).device)
    for p in model.parameters():
        if p.requires_grad and p.ndim > 1:
            spectral_norm += torch.norm(p, p=2)
    return spectral_norm

# Corrected DRO Loss
def compute_dro_loss(losses, groups):
    unique_groups = torch.unique(groups)
    subgroup_losses = []

    for group in unique_groups:
        group_mask = groups == group
        if group_mask.sum() > 0:
            subgroup_loss = losses[group_mask].mean()
            subgroup_losses.append(subgroup_loss)

    dro_loss = torch.max(torch.stack(subgroup_losses))
    return dro_loss

class FusionTrainer(Trainer):
    def __init__(self, model, lambda_irm=0.1, lambda_spectral=0.006, lambda_dro=0.9, *args, **kwargs):
        super().__init__(model, *args, **kwargs)
        self.lambda_irm = lambda_irm
        self.lambda_spectral = lambda_spectral
        self.lambda_dro = lambda_dro

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        input_features = inputs["input_features"].to(model.device)
        labels = inputs["labels"].to(model.device)
        groups = inputs["groups"].to(model.device)

        outputs = model(input_features=input_features, labels=labels)
        logits = outputs.logits

        erm_loss = compute_erm_loss(logits, labels)

        irm_penalty_value = irm_penalty(logits, labels, groups) if model.training else 0.0

        spectral_norm = compute_spectral_norm(model) if model.training else 0.0
        spectral_decoupling_loss = self.lambda_spectral * spectral_norm

        loss_fct = torch.nn.CrossEntropyLoss(reduction="none", ignore_index=-100)
        losses = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
        dro_loss = compute_dro_loss(losses.view(labels.size(0), -1).mean(dim=1), groups) if model.training else 0.0

        total_loss = (
            erm_loss
            + self.lambda_irm * irm_penalty_value
            + spectral_decoupling_loss
            + self.lambda_dro * dro_loss
        )

        return (total_loss, outputs) if return_outputs else total_loss


In [ ]:
# Training setup
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=6,
    weight_decay=0.01,
    save_total_limit=2,
    gradient_accumulation_steps=8,
    fp16=True,
    remove_unused_columns=False,
    logging_strategy="steps",
    logging_steps=10,
    report_to="none",
    max_grad_norm=1.0
)

# Initialize Fusion Trainer
trainer = FusionTrainer(
    model=model,
    args=training_args,
    train_dataset=debiased_train,
    eval_dataset=debiased_val,
    data_collator=DataCollatorForDebiasedSeq2Seq(),
    lambda_irm=0.1,  # Weight for IRM penalty
    lambda_spectral=0.006,  # Weight for Spectral Decoupling
    lambda_dro=0.9  # Weight for DRO
)

# Start training with fusion loss
print("Starting training with fusion loss...")
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting training with fusion loss...


Epoch,Training Loss,Validation Loss
0,8.796300,1.312588
1,8.276200,1.132509
2,7.951100,1.051810
3,7.833500,1.006803
4,7.683600,0.984878
5,7.730700,0.977477


TrainOutput(global_step=390, training_loss=8.23867185543745, metrics={'train_runtime': 2439.8437, 'train_samples_per_second': 5.162, 'train_steps_per_second': 0.16, 'total_flos': 3.087208194048e+17, 'train_loss': 8.23867185543745, 'epoch': 5.9904761904761905})

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import evaluate
import re
import random
from torch.utils.data import DataLoader

# Ensure WER metric is loaded
wer_metric = evaluate.load("wer")

#   - "groups": the gender label (1 for male, 2 for female)
debiased_test = voxpopuli_data["test"].map(lambda ex: apply_debias(ex, P, device="cuda"))
# (Optionally, you can further process debiased_test with your custom create_hf_dataset function.)

class DataCollatorForDebiasedSeq2Seq:
    def __call__(self, features):
        # Process "input_features": each is a 2D array [80, T]
        input_feats = [torch.tensor(f["input_features"], dtype=torch.float32) for f in features]
        max_time = max(feat.shape[1] for feat in input_feats)
        padded_feats = []
        for feat in input_feats:
            t = feat.shape[1]
            if t < max_time:
                pad_amount = max_time - t
                feat_padded = F.pad(feat, (0, pad_amount), "constant", 0.0)
            else:
                feat_padded = feat
            padded_feats.append(feat_padded)
        batch = {"input_features": torch.stack(padded_feats)}

        # Process "labels": assume they are 1D arrays (variable length)
        if "labels" in features[0]:
            labels_list = [torch.tensor(f["labels"], dtype=torch.long) for f in features]
            max_len_labels = max(l.shape[0] for l in labels_list)
            padded_labels = []
            for l in labels_list:
                if l.shape[0] < max_len_labels:
                    pad_amount = max_len_labels - l.shape[0]
                    l_padded = F.pad(l, (0, pad_amount), "constant", -100)
                else:
                    l_padded = l
                padded_labels.append(l_padded)
            batch["labels"] = torch.stack(padded_labels)

        # Process "groups": assume scalar per example
        if "groups" in features[0]:
            batch["groups"] = torch.tensor([f["groups"] for f in features], dtype=torch.long)

        return batch

# Create a data collator instance
data_collator = DataCollatorForDebiasedSeq2Seq()
eval_dataloader = DataLoader(debiased_test, batch_size=4, collate_fn=data_collator)

def normalize_text(text):
    text = text.lower().strip()
    text = text.replace("\n", " ")
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Remove non-alphanumeric characters
    return text

def evaluate_wer_debiased(eval_dataloader, model, processor, device):
    model.eval()  # Set model to evaluation mode

    total_wer, total_male_wer, total_female_wer = 0, 0, 0
    male_count, female_count, total_count = 0, 0, 0

    with torch.no_grad():  # Disable gradients for evaluation
        for batch in eval_dataloader:
            input_features = batch['input_features'].to(device)
            labels = batch['labels'].to(device)
            groups = batch['groups'].to(device)

            # Create an attention mask: assume nonzero values in input_features indicate valid data
            attention_mask = (input_features.sum(dim=1) != 0).long()
            # Generate predictions (adjust max_new_tokens and num_beams as needed)
            generated_ids = model.generate(
                input_features,
                max_new_tokens=256,
                num_beams=1,
                attention_mask=attention_mask,
                language="sl"
            )

            # Decode predictions
            transcriptions = processor.batch_decode(generated_ids, skip_special_tokens=True)

            for idx, (transcription, label_ids, group) in enumerate(zip(transcriptions, labels, groups)):
                hypothesis_text = normalize_text(transcription)

                # Convert label_ids to list and filter out invalid token IDs (padding -100)
                label_ids = label_ids.tolist()
                label_ids = [tid for tid in label_ids if tid >= 0 and tid < processor.tokenizer.vocab_size]

                if label_ids:
                    try:
                        reference_text = normalize_text(processor.tokenizer.decode(label_ids, skip_special_tokens=True))
                    except Exception as e:
                        continue

                    # Compute WER for the sample
                    sample_wer = wer_metric.compute(predictions=[hypothesis_text], references=[reference_text])

                    total_wer += sample_wer
                    total_count += 1

                    if group.item() == 1:  # Male
                        total_male_wer += sample_wer
                        male_count += 1
                    elif group.item() == 2:  # Female
                        total_female_wer += sample_wer
                        female_count += 1

    avg_wer = total_wer / total_count if total_count > 0 else 0
    avg_male_wer = total_male_wer / male_count if male_count > 0 else 0
    avg_female_wer = total_female_wer / female_count if female_count > 0 else 0

    return avg_wer, avg_male_wer, avg_female_wer

# ------------------------------------------------------------------------------
# Step 5: Run Evaluation
# ------------------------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

avg_wer_debiased, avg_male_wer_debiased, avg_female_wer_debiased = evaluate_wer_debiased(
    eval_dataloader, model, processor, device
)

print(f"\nDebiased Combined Average WER: {avg_wer_debiased:.4f}")
print(f"Debiased Male Average WER: {avg_male_wer_debiased:.4f}")
print(f"Debiased Female Average WER: {avg_female_wer_debiased:.4f}")
print(f"Debiased Gender WER Gap: {abs(avg_male_wer_debiased - avg_female_wer_debiased):.4f}")


Debiased Combined Average WER: 0.7624
Debiased Male Average WER: 0.8394
Debiased Female Average WER: 0.6270
Debiased Gender WER Gap: 0.2124
